# How to get started
Kears ist seit kurzem in der Version 3.x und supported nun auch PyTorch (und JAX?!) neben Tnsorflow. Für alle die linux oder mac beutzen ist in diesem Repo ein kleiner bash script angehängt um den start zu vereinfachen. Details wurden in der Präsentation besprochen.

## Warum PyTorch
PyTorch ist bisschen anders als Tensorflow. Es ist vor allem für Researchen gedacht. TF benutzt statische Graphen und PyTorch dynamische Graphen.
Das wichtigste Unterschied ist aber die Syntax. PyTorch hat eine Syntax die für die meisten python entwickler einfacher zum verstehen ist.

## Warum Tensorflow
Tensorflow ist nach wie vor **die** beste Library, wenn es um deployment geht. In der Praxis sollen entwickelte Modelle auch mal an irgendeinem Gerät oder hinter einer API laufen. Tensorflow bietet hier die breiteste und beste Auswahl out of the box ohne größeren Aufwand.
 - Tensorflow serving (docker basiert)
 - TPU support (RPI)
 - Tensorflow Lite für SoCs
 - Tensorflow JS für Webentwickler

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow" # or "jax" or "torch"
import keras
print(keras.backend.backend())

tensorflow


## Sequential API
entweder gibt man dem model am Anfang eine Liste von Layers oder man benutzt die `.add(layer)` methode

In [3]:
import keras.layers as l

In [4]:
layers = [
    l.Input((200,)),
    l.Dense(1024, activation="relu"),
    l.Dense(296, activation="relu"),
    l.Dense(10),
    l.Softmax()  # man kann auch  Activations als Layers instanzieren
]

model = keras.Sequential(layers=layers)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.CategoricalCrossentropy(),  # multiclass classifications
    metrics=[
        keras.metrics.F1Score(),
        keras.metrics.Accuracy(),
    ],
)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1024)           │       205,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 296)            │       303,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         2,970 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Softmax)               │ (None, 10)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 512,194 (1.95 MB)

 Trainable params: 512,194 (1.95 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
model2 = keras.Sequential()
model2.add(l.Input((200,)))
model2.add(l.Dense(1024, activation="relu"))
model2.add(l.Dense(296, activation="relu"))
model2.add(l.Dense(10))
model2.add(l.Softmax())

model2.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.CategoricalCrossentropy(),  # multiclass classifications, one hot encoding
    metrics=[
        keras.metrics.F1Score(),
        keras.metrics.Accuracy(),
    ],
)

model2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 1024)           │       205,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 296)            │       303,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         2,970 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax_1 (Softmax)             │ (None, 10)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 512,194 (1.95 MB)

 Trainable params: 512,194 (1.95 MB)

 Non-trainable params: 0 (0.00 B)

# Functional API

In [10]:
inputs = l.Input((200,))
features = l.Dense(1024, activation="relu") (inputs)
features = l.Dense(296, activation="relu") (features)
features = l.Dense(10) (features)
outs = l.Softmax() (features)

model_functional = keras.Model(inputs=inputs, outputs=outs, name="model_functional")

model_functional.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.CategoricalCrossentropy(),  # multiclass classifications, one hot encoding
    metrics=[
        keras.metrics.F1Score(),
        keras.metrics.Accuracy(),
    ],
)

model_functional.summary()

Model: "model_functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1024)           │       205,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 296)            │       303,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 10)             │         2,970 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax_5 (Softmax)             │ (None, 10)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 512,194 (1.95 MB)

 Trainable params: 512,194 (1.95 MB)

 Non-trainable params: 0 (0.00 B)

# Model Subclassing
Im Buch wird auch noch die Möglichkeit gezeigt, wie man eine subklasse von Model erstellen, durch vVererbung. Tatsächlich ist diese Methode aber weniger mächtig als
die Funtional API da man nicht auf die einezelnen Ausgaben der Layer _auserhalb_ von dem Forwardpass zugreifen kann. Das ist limietierend insofern man dies braucht. 

Aus meiner Praxiserfahrung wird das subclassing fast nie verwendet, da auch der Setup wesentlich anders ist als in den beiden anderen fällen.

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers as l

class Model3(keras.Model):
    def __init__(self):
        super(Model3, self).__init__()
        self.input_layer = l.InputLayer(input_shape=(200,))
        self.dense1 = l.Dense(1024, activation="relu")
        self.dense2 = l.Dense(296, activation="relu")
        self.dense3 = l.Dense(10)
        self.softmax = l.Softmax()

    def call(self, inputs):
        x = self.input_layer(inputs)
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        outputs = self.softmax(x)
        return outputs

# Create an instance of the model
model3 = Model3()

/Users/martinb/miniconda3/envs/tf-backend/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [8]:
model3.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.CategoricalCrossentropy(),  # multiclass classifications, one hot encoding
    metrics=[
        keras.metrics.F1Score(),
        keras.metrics.Accuracy(),
    ],
)
model3.summary()

Model: "model3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax_3 (Softmax)             │ ?                      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)